In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [13]:
batch = 64
num_classes = 10
learning_rate = 0.001
epoch = 10

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
    train_dataset = torchvision.datasets.MNIST(root = './data',
                                               train = True,
                                               transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                               download = True)
    
    
    test_dataset = torchvision.datasets.MNIST(root = './data',
                                              train = False,
                                              transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                              download=True)
    
    
    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                               batch_size = batch,
                                               shuffle = True)
    
    
    test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                               batch_size = batch,
                                               shuffle = True)

In [9]:
class LeNet(nn.Module):
        def __init__(self, num_classes):
            super(LeNet, self).__init__()
            self.layer1 = nn.Sequential(
                nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(6),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 2, stride = 2))
            self.layer2 = nn.Sequential(
                nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 2, stride = 2))
            self.fc = nn.Linear(400, 120)
            self.relu = nn.ReLU()
            self.fc1 = nn.Linear(120, 84)
            self.relu1 = nn.ReLU()
            self.fc2 = nn.Linear(84, num_classes)
            
        def forward(self, x):
            out = self.layer1(x)
            out = self.layer2(out)
            out = out.reshape(out.size(0), -1)
            out = self.fc(out)
            out = self.relu(out)
            out = self.fc1(out)
            out = self.relu1(out)
            out = self.fc2(out)
            return out

In [11]:
    model = LeNet(num_classes).to(device)
    
    #Setting the loss function
    cost = nn.CrossEntropyLoss()
    
    #Setting the optimizer with the model parameters and learning rate
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    #this is defined to print how many steps are remaining when training
    total_step = len(train_loader)

In [14]:
    total_step = len(train_loader)
    for x in range(epoch):
        for i, (images, labels) in enumerate(train_loader):  
            images = images.to(device)
            labels = labels.to(device)
            
            #Forward pass
            outputs = model(images)
            loss = cost(outputs, labels)
            #Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i+1) % 400 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
.format(x+1, epoch, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.1276
Epoch [1/10], Step [800/938], Loss: 0.0520
Epoch [2/10], Step [400/938], Loss: 0.1094
Epoch [2/10], Step [800/938], Loss: 0.1610
Epoch [3/10], Step [400/938], Loss: 0.0155
Epoch [3/10], Step [800/938], Loss: 0.0526
Epoch [4/10], Step [400/938], Loss: 0.0160
Epoch [4/10], Step [800/938], Loss: 0.0263
Epoch [5/10], Step [400/938], Loss: 0.0825
Epoch [5/10], Step [800/938], Loss: 0.1245
Epoch [6/10], Step [400/938], Loss: 0.0140
Epoch [6/10], Step [800/938], Loss: 0.0908
Epoch [7/10], Step [400/938], Loss: 0.0013
Epoch [7/10], Step [800/938], Loss: 0.0244
Epoch [8/10], Step [400/938], Loss: 0.0037
Epoch [8/10], Step [800/938], Loss: 0.0004
Epoch [9/10], Step [400/938], Loss: 0.0006
Epoch [9/10], Step [800/938], Loss: 0.0007
Epoch [10/10], Step [400/938], Loss: 0.1925
Epoch [10/10], Step [800/938], Loss: 0.0001


In [20]:
# Test the model
      
with torch.no_grad():
    correct = 0
    total = 0
    
for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 99.08 %
